In [1]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import wrds
import psycopg2

In [2]:
db = wrds.Connection(wrds_username="thomas_bienaime")
#db.create_pgpass_file()

Loading library list...
Done


In [3]:
# we download the stocks returns 
stocks_returns_raw = db.raw_sql("""
                      select distinct a.permno, a.date,
                      a.ret
                      from crsp.msf as a
                      left join crsp.msenames as b
                      on a.permno=b.permno
                      and b.namedt<=a.date
                      and a.date<=b.nameendt
                      where a.date between '01/01/1980' and '12/31/2021'
                      and b.exchcd = 1 or b.exchcd = 2
                      and b.shrcd = 10 or b.exchcd = 11
                      """, date_cols=['date']) 


#we download the risk free rate
risk_free=db.raw_sql("select  mcaldt,tmytm "
           "from crsp.tfz_mth_rf "           
            "where kytreasnox = 2000001 "
           "and mcaldt>='1980-01-01'"
            "and mcaldt<='2021-12-31'", date_cols=['mcaldt'])
risk_free = risk_free.rename(
    columns ={"mcaldt" : "date", "tmytm" :"risk_free" })
risk_free = risk_free.set_index("date")
# downlad index returns
market_index=db.raw_sql("select  date,vwretd "
           "from crsp.msi "
           "where date>='1980-01-01'"
            "and date<='2021-12-31'", date_cols=['date'])
market_index = market_index.set_index("date")
#we tranform the returns of the risk free rate
risk_free['risk_free']=np.exp(risk_free['risk_free']/(12*100))-1

In [4]:
stocks_returns_raw

,permno,date,ret
0,27983.0,1987-10-30,-0.276899
1,29161.0,1966-03-31,-0.136364
2,42796.0,2019-10-31,0.028369
3,38578.0,1993-10-29,-0.041916
4,44003.0,1973-03-30,0.017544
...,...,...,...
302925,88817.0,2003-10-31,0.080036
302926,89363.0,2004-03-31,-0.013444
302927,85059.0,2002-03-28,0.058974
302928,62930.0,2006-09-29,0.041789


In [5]:
number_observations=stocks_returns_raw.groupby(by=["permno"]).count()
stocks_returns_pemno= number_observations[number_observations["ret"]>=504].index.sort_values()

In [6]:
# put all data into a dataframe indexed by their permno and date, and each return is on the right date
risk_free = risk_free.sort_index()
stocks_ret_matrix = pd.DataFrame(index = risk_free.index, data = risk_free)
stocks_ret_matrix = pd.merge(stocks_ret_matrix, market_index, left_index=True, right_index= True, suffixes=("",""))
for permno in stocks_returns_pemno:
    display()
    stocks_ret_matrix = pd.merge(stocks_ret_matrix,
                  stocks_returns_raw[stocks_returns_raw["permno"]==permno]
                                 .set_index("date")["ret"].rename( permno ),
                  left_index = True, right_index = True, suffixes=("",""))
stocks_ret_matrix

,risk_free,vwretd,10065.0,10516.0,10866.0,10874.0,10890.0,11308.0,11404.0,11674.0,...,60206.0,60580.0,60599.0,60628.0,60687.0,60943.0,61188.0,61313.0,61399.0,62922.0
date,,,,,,,,,,,,,,,,,,,,,
1980-01-31,0.009816,0.065607,0.099216,0.182186,0.053922,0.071028,0.004785,0.018116,-0.039381,0.010101,...,-0.079070,-0.275132,-0.060606,0.278820,0.064865,0.067416,0.055556,0.163522,-0.055172,0.077714
1980-02-29,0.011412,0.001015,-0.029126,-0.032877,-0.088372,0.061947,-0.080952,-0.064057,-0.060773,-0.020000,...,-0.047475,-0.036496,-0.011613,-0.014675,-0.111675,0.104842,0.002105,-0.085405,-0.044118,-0.062321
1980-03-31,0.012621,-0.120224,-0.000400,-0.187943,-0.040816,-0.141667,-0.067012,-0.025399,0.005882,-0.048980,...,-0.149733,-0.115455,-0.016667,-0.195745,-0.274286,-0.163462,-0.215054,-0.136905,-0.153846,-0.065382
1980-04-30,0.007728,0.052287,-0.010101,0.100437,-0.010870,-0.147573,-0.031776,0.055556,0.140351,0.166667,...,0.044025,-0.078261,0.067797,-0.134921,0.086614,0.034483,0.178082,-0.089655,0.101818,0.089209
1980-05-30,0.006003,0.060090,0.061224,0.076984,0.131868,0.162791,0.042471,0.015038,0.042872,0.000000,...,0.034217,0.056604,0.083810,0.091743,0.028986,0.056000,0.073488,0.115152,0.125000,0.032650
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-31,0.000034,0.027147,0.037830,0.010884,-0.006063,-0.068918,0.083221,-0.012625,0.033279,0.025742,...,0.037618,-0.033409,0.006415,-0.050938,0.090521,0.007495,0.013942,0.026892,0.058124,-0.038704
2021-09-30,0.000038,-0.042244,-0.048077,0.000167,-0.093534,-0.016517,0.038414,-0.060735,-0.037906,-0.064858,...,-0.071127,-0.113971,0.007317,-0.171817,-0.058794,-0.064683,-0.048784,-0.152620,-0.005052,-0.005271
2021-10-29,0.000042,0.064658,0.071212,0.070488,0.037804,-0.022882,0.017104,0.074328,0.038711,0.014681,...,-0.027375,-0.017221,-0.042776,0.074057,0.085592,0.044826,0.035437,0.045288,0.156561,0.151408


In [7]:
betas = {}
for permno in stocks_returns_pemno:
    cov_matrix = stocks_ret_matrix[["vwretd",permno]].cov()
    betas[permno] = cov_matrix.loc[permno,'vwretd']/cov_matrix.loc["vwretd", "vwretd"]
betas

{10065.0: 0.8620601755947288,
 10516.0: 0.7662388293935742,
 10866.0: 1.2699751847029448,
 10874.0: 1.6691240082589014,
 10890.0: 1.7700823652601443,
 11308.0: 0.5979317902810123,
 11404.0: 0.1536383537332066,
 11674.0: 0.34575098657777203,
 11762.0: 1.0588904048207661,
 11850.0: 0.6779778615526149,
 12036.0: 1.0677715142478117,
 12052.0: 0.8994401638801465,
 12060.0: 1.1195758408414014,
 12431.0: 1.3250993880135842,
 12490.0: 0.9050095020252187,
 12570.0: 1.0259490498801156,
 12781.0: 0.2714783191153831,
 13303.0: 1.073883750567117,
 13610.0: 1.1113813308381868,
 13688.0: 0.4218527450588317,
 13901.0: 0.5926944407404541,
 13928.0: 0.9992432587868318,
 13936.0: 1.2949557832267322,
 14277.0: 1.2840996764800385,
 14541.0: 0.8250884072507504,
 14795.0: 1.3149616619852722,
 14816.0: 0.6287623455060966,
 15069.0: 1.2441694323548969,
 15202.0: 0.7869857639393698,
 15456.0: 1.0078137601412256,
 15720.0: 0.36706570272544187,
 16555.0: 0.6708788802197475,
 16600.0: 0.320342606488395,
 16678.0: 